In [ ]:
using DataFrames, DelimitedFiles, DataStructures
using GLPKMathProgInterface
using JuMP, StatsBase, CSV, LinearAlgebra, Distributions, Random
using SCIP, GLPK, BenchmarkTools

## Function to clean inputs


In [ ]:
function clean_input(year, week, version, source)
    cd(string("/home/sagemaker-user/dfs-nfl/Data/Projections/", source, "/", year))
    offense = CSV.read(string("Week ", week, "/main_slate_v", version, ".csv"), copycols=true, DataFrame)

    if source ==  "SaberSim"
        rename!(offense, Symbol(string("Name")) => :Player)
        rename!(offense, Symbol(string("Pos")) => :"DK Position")
        rename!(offense, Symbol(string("Opp")) => :Opponent)
        rename!(offense, Symbol(string("dk_85_percentile")) => :Projection)
        rename!(offense, Symbol(string("DFS ID")) => :DKSlateID)
        offense = offense[:, [
                :Player,
                Symbol("DK Position"),
                :Team,
                :Opponent,
                :Salary,
                :Projection,
                :DKSlateID
            ]]
        offense."DK Position" .= replace.(offense."DK Position", r",FLEX" => "")
    elseif source == "ETR"
        rename!(offense, Symbol(string("DK Salary")) => :Salary)
        rename!(offense, Symbol(string("DK Ceiling")) => :Projection)
        rename!(offense, Symbol(string("DK Ownership")) => :Ownership)
    elseif source == "Stokastic"
        rename!(offense, Symbol(string("Name")) => :Player)
        rename!(offense, Symbol(string("Position")) => :"DK Position")
        rename!(offense, Symbol(string("Fpts")) => :Projection)
    end




    offense = offense[completecases(offense), :]

 
    #cut off players that have projected points less than 5
    offense = offense[(offense."Projection".>=9) .| (offense."DK Position".== "DST"),:]
    #remove injured players

    #scores_o = copy(offense)
    #global scores_d = copy(defense)

    global num_offense = size(offense)[1]

    
    QB_indicies = Vector{Int}()
    RB_indicies = Vector{Int}()
    WR_indicies = Vector{Int}()
    TE_indicies = Vector{Int}()
    FLEX_indicies = Vector{Int}()
    DST_indicies = Vector{Int}()


   
    for i =1:num_offense
        if offense."DK Position"[i] == "QB"
            append!(QB_indicies, i)
        elseif offense."DK Position"[i] == "RB"
            append!(RB_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "WR"
            append!(WR_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "TE"
            append!(TE_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "DST"
            append!(DST_indicies, i)
        else
            append!(FLEX_indicies, i)
        end
    end
    
    return offense, QB_indicies, RB_indicies, WR_indicies, TE_indicies, FLEX_indicies, DST_indicies
        
        
    
end

In [ ]:
function calc_cor(player_matrix, temp_teams, player_cor, i, j)
    if ismissing(temp_teams[i,2])
        player_matrix[i,j] .= 0
    elseif ismissing(temp_teams[j,2])
        player_matrix[i,j] = 0
    elseif temp_teams[i,2]==temp_teams[j,2]
        try
            orig_a = player_cor[:,i]
            orig_b = player_cor[:,j]

            a = orig_a[(ismissing.(orig_b).==false) .& (ismissing.(orig_a).==false),1]
            b = orig_b[(ismissing.(orig_a).==false) .& (ismissing.(orig_b).==false),1]
            
            if (i ==1) & (j == 47)
                println(a, b)
            end
            player_matrix[i,j] = cor(a, b)
        catch err
            player_matrix[i,j] = 0
        end
    else
        player_matrix[i,j] = 0
    end
    return player_matrix[i,j]
end

In [ ]:
function calc_var(player_matrix, temp_teams, player_cor, i)

    orig_a = player_cor[:,i]
    #a = orig_a[(ismissing.(orig_a).==false),1]
    a = skipmissing(orig_a)

    
    println(a)
    try
        variance = var(a)
    catch err
        variance = 0    
    end
    
    return variance
end

In [ ]:
function get_variance(from, to, v, max_score, year, source)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    offense2 = get_stripped_offense(from)
    offense2 = offense2[:,[:Player_Name,:Actual_FP]]
    temp = offense2
    
    for i=from+1:to-1
        offense2 = get_stripped_offense(i)
        offense2 = offense2[:,[:Player_Name,:Actual_FP]]

   
        temp = outerjoin(temp, offense2, on = :Player_Name, makeunique=true)
        
    end
    #print(temp)
    
    #temp = Tables.matrix(temp[:,2:end])
    #print(temp)
    #print(temp)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    # current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(to, v);
    current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input(year, to, v, source);
    current_offense."Player" .= replace.(current_offense."Player", r"-" => "")
    current_offense."Player" .= replace.(current_offense."Player", r"\." => "")
    current_offense."Player" .= replace.(current_offense."Player", r"'" => "")
    transform!(current_offense, :Player => ByRow(lowercase) => :Player)
    rename!(current_offense, Symbol(string("Player")) => :Player_Name)
   
   #print(current_offense) 
    
    var_array = []
    
    for i=1:length(current_offense[:,1])
        points = []
        #println(current_offense[i,1])
        #println(Matrix(filter(row -> row.Player_Name == current_offense[i,1], temp)))
        
        #println(temp)
        
        if current_offense[i,1] in temp[:,1]
            #println(current_offense[i,1])
            points_df = filter(row -> row.Player_Name == current_offense[i,1], temp)
            #println(points_df)
            for j = 2:(to-from)+1
                if ismissing(points_df[1,j]) == false
                    if Float64.(points_df[1,j]) != 0
                        append!(points, points_df[1,j])
                    end
                end
            end
            
            #println(current_offense[i,1], points)
        else            
            points = 0
        end
        
        if length(points) > 0
            variance = var(points)
        else
            variance = 0
        end
        
        if variance > 0
            append!(var_array, variance)
        else
            append!(var_array, 0)
        end
            
        #println(var_array)
    end
    
    #println(var_array)

    b = Float64.(var_array)
    
    scale = maximum(b) / max_score
    
    b = b ./ scale
    
    #print(b)
    
    return b
end


In [ ]:
# var_boost = get_variance(6,12,4, 33);

In [ ]:
# var_boost[1]

In [ ]:
function get_stripped_offense(week)
    cd(string("/home/sagemaker-user/dfs-nfl/Data/Actuals/2022-2023/Week ", week))
    offense = CSV.read("Offense.csv", copycols=true, DataFrame)
    rename!(offense, Symbol(string("Proj FP")) => :Projection)
    #offense = offense[offense[:,:Projection].>=3.01,:]
    
    offense."Player Name" .= replace.(offense."Player Name", r"-" => "")
    offense."Player Name" .= replace.(offense."Player Name", r"\." => "")
    offense."Player Name" .= replace.(offense."Player Name", r"'" => "")
    
    rename!(offense, Symbol(string("Player Name")) => :Player_Name)
    rename!(offense, Symbol(string("Actual FP")) => :Actual_FP)
    transform!(offense, :Player_Name => ByRow(lowercase) => :Player_Name)

    return offense
end

In [ ]:
function get_correlation_matrix(from, to, v, year, source)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")

    for i=from:to-1
        offense2 = get_stripped_offense(i)
        offense2 = offense2[:,[:Player_Name,:Actual_FP]]


        #print(i)
        if i==from
            temp = offense2
        else
            temp = outerjoin(temp, offense2, on = :Player_Name, makeunique=true)
        end
    end
    #print(temp)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    # current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(to, v);
    current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input(year, to, v, source);
    current_offense."Player" .= replace.(current_offense."Player", r"-" => "")
    current_offense."Player" .= replace.(current_offense."Player", r"\." => "")
    current_offense."Player" .= replace.(current_offense."Player", r"'" => "")
    transform!(current_offense, :Player => ByRow(lowercase) => :Player)
    rename!(current_offense, Symbol(string("Player")) => :Player_Name)


    current_offense = current_offense[:,[:Player_Name,:Team]]
    #temp = temp[:,[:Player_Name,:Actual_FP]]

    #print(temp)
    #print(current_offense)
    #print(temp)
    #gold skeleton key
    temp = leftjoin(current_offense, temp, on = :Player_Name, makeunique=true)


    #temp = leftjoin(current_offense, temp, on = [:Player_Name])
    #print(temp2)
    #println(temp)
    temp = temp[:,setdiff(names(temp), [:Team])]
    temp_teams = current_offense

    temp = Matrix(temp)
    #print(temp)
    temp = permutedims(temp)
    #print(temp)


    player_list = temp[1,:]
    player_cor = temp[3:end,:]
    temp_teams = temp_teams[:,[:Player_Name,:Team]]
    #println(player_cor)



    n = size(temp_teams)[1]
    player_matrix = zeros(n,n)
    #temp_teams
    for i=1:n,j=1:n
        
        #if i in FLEX_i && j in FLEX_i && j in opp_team[i]
         #   player_matrix[i,j] = (cOffense[j,:Projection]/(9*10))
        if i in DST_i || j in DST_i
            player_matrix[i,j] = 0
        elseif i in QB_i && j in RB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        elseif j in QB_i && i in RB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j) 
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        elseif i in QB_i || j in QB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
        else
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        end
        if isnan(player_matrix[i,j])
            player_matrix[i,j] = 0
        end
    end

    for i=1:n
        player_matrix[i,i] = 1
    end
    player_matrix[player_matrix.<-0.2] .= -100;
    return player_matrix
end

In [ ]:
function get_bring_back(week, QB_i, offense)
    
    bring_back = Vector{Vector{Int}}()

    for i = 1:length(QB_i)
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[j, :Team] == offense[QB_i[i], :Opponent]) && (j in FLEX_i)
                append!(temp, j)

            end
        end
        push!(bring_back, temp)   
    end
    
    return bring_back
end

In [ ]:
function get_opposing_defense(week, QB_i, offense)
    
    opp_defense = Vector{Vector{Int}}()

    for i = 1:length(QB_i)
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[QB_i[i], :Team] == offense[j, :Opponent]) && (j in DST_i)
                append!(temp, j)

            end
        end
        push!(opp_defense, temp)   
    end
    
    return opp_defense
end

In [ ]:
function get_opposing_team(week, offense)
    
    opp_team = Vector{Vector{Int}}()

    for i = 1:size(t)[1]
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[i, :Team] == offense[j, :Opponent]) && i in FLEX_i && j in FLEX_i
                append!(temp, j)

            end
        end
        push!(opp_team, temp)   
    end
    
    return opp_team
end

In [ ]:
function get_games(df)
    unq = unique(df[:,:Team])
    num_games = length(unq)
    game = Vector{Vector{Int}}()

    for i = 1:num_games
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if j in FLEX_i || j in DST_i
                if (df[j, :Team] == unq[i]) || (df[j, :Opponent] == unq[i]) 
                    append!(temp, j)

                end
            end
        end
        push!(game, temp)   
    end
    
    return game, num_games
    
end

In [ ]:
function get_teams(df)
    unq = unique(df[:,:Team])
    num_teams = length(unq)
    team = Vector{Vector{Int}}()

    for i = 1:num_teams
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            #if j in FLEX_i || j in DST_i
                if (df[j, :Team] == unq[i]) 
                    append!(temp, j)

                end
           # end
        end
        push!(team, temp)   
    end
    
    return team, num_teams
    
end

In [ ]:
function get_idx(t)
    team_idx = Vector{Vector{Int}}()

    for i=1:size(t)[1]

        temp = Vector{Int}()

        for j=1:size(t)[2]
            if i<j
                if t[i,j] != 0
                    append!(temp, j)
                end
            end    
        end

        push!(team_idx, temp)   

    end




    #####################

    team_idx_cross = Vector{Vector{Int}}()

    for i=1:size(t)[1]

        temp = Vector{Int}()

        for j=1:size(t)[2]
            #if i<=j
                if t[i,j] > 0
                    append!(temp, j)
                end
            #end    
        end

        push!(team_idx_cross, temp)   
    end
    
    return team_idx, team_idx_cross
end

In [ ]:
# function late_swap_ids(week, new_version, late_teams)
#     df, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(week, new_version)

#     ids = readdlm(string("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week ", week, "/dk_ids3.csv"), ',', Int)
    
#     all_teams = []
#     for i = 1:length(ids[:,1])
#         team_i = []
#         for j = 1:9            
#             row = findall(x -> x == ids[i,j], df."DKSlateID")
#             if length(row) > 0
#                 if (df[row[1], :Team] in late_teams) == false 
#                     team_i = push!(team_i, row[1])
#                 end
#             end
#         end
#         all_teams = push!(all_teams, team_i)
#     end
    
#     return all_teams
# end

In [ ]:
late_teams = ["DEN", "CIN", "ATL", "GB", "SF", "BAL"]

## required input cleanings

In [ ]:
year = "2022-2023"
source = "SaberSim" # "Stokastic"
week = 1
version = 1
max_score = 38
variance_start_week = -3
correlation_start_week = week -4

cOffense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input(year, week, version, source);

#QB, RB, WR, TE, num_teams, num_offense, num_defense, offense, defense, offense_opp, offense_teams = clean_input(week);

t = get_correlation_matrix(correlation_start_week, week, version, year, source)



cNum_offense = length(cOffense[:,1])
#cNum_teams = length(cDefense[:,1])
num_QB = length(QB_i)
num_RB = length(RB_i)
num_WR = length(WR_i)
num_TE = length(TE_i)
num_DST = length(DST_i)
num_FLEX = length(FLEX_i)


var_boost = get_variance(variance_start_week, week, version, max_score, year, source);

In [ ]:
# get a player that's opposing your QB
bring_back = get_bring_back(week, QB_i, cOffense);

In [ ]:
opp_team = get_opposing_team(week, cOffense);

In [ ]:
# get dataframe indexes of players on same team
# cross is indexes of players on opposing team
team_idx, team_idx_cross = get_idx(t);

In [ ]:
# getting games and teams that are playing for the selected slate
game_stack, num_games = get_games(cOffense);
team_stack, num_teams = get_teams(cOffense);

## make optimization

In [ ]:
function makeTeamX(num, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, var_boost)#num is what team it is from 1-100, X is current teams
    m1 = Model(SCIP.Optimizer)
    set_silent(m1)
    #GLPK.Optimizer
    M = 100
    max_overlap = 5.1
    max_same_player = 40
    @variable(m1, x[1:cNum_offense], Bin) #pick players
    #@variable(m1, def[1:cNum_teams], Bin) #pick players
    #@variable(m1, PB[1:cNum_offense], Bin) #player boost
    
    
    #@constraint(m1, [i=1:length(FLEX_i)], PB[FLEX_i[i]] <= sum(x[opp_team[i][j]] for j = 1:length(opp_team[i])))
    
    #@constraint(m1, [i=1:length(QB_i)], PB[QB_i[i]] == 0)
    #@constraint(m1, [i=1:length(DST_i)], PB[DST_i[i]] == 0)

    #@variable(m1, G[1:num_games], Bin)
    #@constraint(m1,[i=1:num_games], 1.5 * G[i] <= sum(x[game_stack[i][j]] for j=1:length(game_stack[i])))
    #@constraint(m1, sum(G[i] for i=1:num_games) >= 2)
    #need to make num_games and game_stack, try using uniques to get it
    
    @variable(m1, u>=0) #pick players


    @objective(m1, Max, u)

    @constraint(m1, sum(x[i]*rand_vec[i] for i=1:cNum_offense) +  # cOffense[i, :Projection] for i=1:cNum_offense) + 
                    (sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) * 9) +
                        (sum(x[i]*var_boost[i] for i=1:cNum_offense) *0.03) >= u)
                    #sum(0.1 * x[i] * x[opp_def[i][j]] for i=1:cNum_offense for j=1:length(opp_def[i])) +
                    #sum(0.1*PB[i]*cOffense[i, :Projection] for i=1:cNum_offense) +
                    

    @constraint(m1, sum(x[QB_i[i]] for i=1:num_QB) == 1)
    @constraint(m1, sum(x[RB_i[i]] for i=1:num_RB) >= 2)
    @constraint(m1, sum(x[WR_i[i]] for i=1:num_WR) >= 3)
    @constraint(m1, sum(x[TE_i[i]] for i=1:num_TE) >= 1)
    @constraint(m1, sum(x[DST_i[i]] for i=1:num_DST) == 1)
    @constraint(m1, sum(x[FLEX_i[i]] for i=1:num_FLEX) == 7)
    @constraint(m1,sum(x[i]*cOffense[i, :Salary] for i=1:cNum_offense) <= 50000)
    @constraint(m1, sum(x[i] for i=1:cNum_offense) == 9)
                        
    # @constraint(m1,sum(x[i]*cOffense[i, :Ownership] for i=1:cNum_offense) <= 200)


    
    
    #@constraint(m1, sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) >= 1.5)
    
    #bring back
                    
    @constraint(m1,[i=1:length(QB_i)], x[QB_i[i]] - sum(x[bring_back[i][j]] for j = 1:length(bring_back[i])) <= 0)
                        
    #team stack          
    @constraint(m1,[i=1:num_teams], sum(x[team_stack[i][j]] for j=1:length(team_stack[i])) <= 4)


    
    #@constraint(m1, sum(def[i] for i=1:cNum_teams) == 1)


    if num != 1
        #for i=1:num
        @constraint(m1,[i=1:num-1], sum(t[X[i][k],team_idx_cross[X[i][k]][j]] * x[team_idx_cross[X[i][k]][j]] for k = 1:9 for j=1:length(team_idx_cross[X[i][k]])) <= 6.9)
        #end
                        
        #@constraint(m1,[i=1:num-1], sum(x[X[i][k]] for k = 1:8) <= max_overlap)


    end 
                                        
    optimize!(m1)
    return(JuMP.value.(x))
end

In [ ]:
function makeAllTeams(count, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, var_boost) #count is how many teams you want
    
    rand_vec = rand(cNum_offense,1)/25 .+1
    temp = cOffense[:, :Projection]
    rand_vec = rand_vec .* temp
    x1 = makeTeamX(1, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, var_boost);

    B = Vector{Int}()
    for i=1:length(x1)
        if x1[i] > 0.5
            append!(B,i)
        end
    end
    X = push!(X, B)
    
    for i = 2:count
        rand_vec = rand(cNum_offense,1)/25 .+1
        temp = cOffense[:, :Projection]
        rand_vec = rand_vec .* temp
        @time x1 = makeTeamX(i, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, var_boost);
        B = Vector{Int}()
        for i=1:length(x1)
            if x1[i] > 0.5
                append!(B,i)
            end
        end
        
        X = push!(X, B)
        #print(X)
        #D = hcat(D, d1)
        
        println(i)

    end

    return(X)
end

In [ ]:
X = Vector{Vector{Int}}()

@time all_teams = makeAllTeams(10, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, var_boost);

In [ ]:
num_teams_making = 150


offense = cOffense
lineup2 = ""
#outfile = open(path_to_output, "w")
print_team = []



    for i = 1:num_teams_making
        lineup = ["" "" "" "" "" "" "" "" ""]
        skip = []


        for j =1:9
        
            if string(offense[all_teams[i][j],2]) in late_teams
                skip = push!(skip, all_teams[i][j])
            else
                    if all_teams[i][j] in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[all_teams[i][j],1])
                        end
                    elseif all_teams[i][j] in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[all_teams[i][j],1])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[all_teams[i][j],1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],1])
                        end
                    elseif all_teams[i][j] in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[all_teams[i][j],1])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[all_teams[i][j],1])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[all_teams[i][j],1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],1])
                        end
                    elseif all_teams[i][j] in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[all_teams[i][j],1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],1])
                        end
                    elseif all_teams[i][j] in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[all_teams[i][j],1])
                        end
                    end
                #end

            end
      end
    #println(skip)
        for k in skip
                    if k in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[k,1])
                        end
                    elseif k in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[k,1])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[k,1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,1])
                        end
                    elseif k in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[k,1])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[k,1])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[k,1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,1])
                        end
                    elseif k in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[k,1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,1])
                        end
                    elseif k in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[k,1])
                        end
                    end
                #end

        end

       print_team = push!(print_team, lineup)
    end
   #close(outfile)
#end

In [ ]:
cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 15/")


writedlm( "test_teams_200exp.csv",  print_team, ',')

In [ ]:
print_team

In [ ]:
print_team[1]

In [ ]:
for i=1:9
    println(offense[all_teams[10][4],1])
end

# correlation check

In [ ]:
corr = 0
team_num = 0
team = "TEN"


for i = 1:length(offense[:,1])
    for j = 1:length(offense[:,1])
        #if all_teams[team_num][i]< all_teams[team_num][j] 
                if offense[i, :Team] == team && offense[j, :Team] == team
            
                #corr = t[all_teams[team_num][i],all_teams[team_num][j]]
                    println(string(offense[i,:Player])," + ", string(offense[j,:Player]), " = ", t[i,j])

                end
            
        #end
    end
end

In [ ]:
length(offense[:,1])

In [ ]:
corr = 0
team_num = 1
team = "NE"


for i = 1:9
    for j = 1:9
        if all_teams[team_num][i]< all_teams[team_num][j] 
     #corr = t[all_teams[team_num][i],all_teams[team_num][j]]
            println(string(offense[all_teams[team_num][i],1])," + ", string(offense[all_teams[team_num][j],1]), " = ", t[all_teams[team_num][i],all_teams[team_num][j]])


        end
    end
end

## player frequnecies

In [ ]:
players = []

for i = 1:num_teams_making
    for j =1:9
        #if print_team[i][j] in QB_i
            push!(players, print_team[i][j])
        #end
    end
end

frequency = []
for i in players
    
    
end

In [ ]:
c = counter(players)

r = hcat(collect(keys(c)),collect(values(c)))

r[sortperm(r[:, 2], rev=true), :]

In [ ]:

cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 15/")



writedlm( "exposure_v3.csv",  r, ',')

## Print money/points

In [ ]:
num_teams_making = 150
num = 9 #column

#5 = salary
#6 = regular projection
#7 = DK value
#8 = ownership
#9 = DK ID
#10 = ceiling projection



offense = cOffense
lineup2 = ""
#outfile = open(path_to_output, "w")
print_team = []



    for i = 1:num_teams_making
        lineup = ["" "" "" "" "" "" "" "" ""]
        skip = []


        for j =1:9
        
            if string(offense[all_teams[i][j],2]) in late_teams
                skip = push!(skip, all_teams[i][j])
            else
                    if all_teams[i][j] in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[all_teams[i][j],num])
                        end
                    elseif all_teams[i][j] in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[all_teams[i][j],num])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[all_teams[i][j],num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],num])
                        end
                    elseif all_teams[i][j] in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[all_teams[i][j],num])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[all_teams[i][j],num])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[all_teams[i][j],num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],num])
                        end
                    elseif all_teams[i][j] in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[all_teams[i][j],num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],num])
                        end
                    elseif all_teams[i][j] in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[all_teams[i][j],num])
                        end
                    end
                #end

            end
      end
    #println(skip)
        for k in skip
                    if k in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[k,num])
                        end
                    elseif k in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[k,num])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[k,num])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[k,num])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[k,num])
                        end
                    end
                #end

        end

       print_team = push!(print_team, lineup)
    end
   #close(outfile)
#end

In [ ]:
cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 15/")



writedlm( "dk_ids.csv",  print_team, ',')

In [ ]:
scores = []

for i =1:150
    println(sum(parse.(Float64, print_team[i])))
    append!(scores, sum(parse.(Float64, print_team[i])))
end

In [ ]:
function clean_print_teams(offense, week)
    DFN_offense = get_stripped_offense(week)
    
    offense."Player" .= replace.(offense."Player", r"-" => "")
    offense."Player" .= replace.(offense."Player", r"\." => "")
    offense."Player" .= replace.(offense."Player", r"'" => "")
    
    transform!(offense, :Player => ByRow(lowercase) => :Player)
    
    #offense[!, :Actual_FP] = 0



    for i = 1:length(offense[:,1])
        for j = 1:length(DFN_offense[:,1])
            if offense[i, :Player] == DFN_offense[j, :Player_Name]
               offense."DK Value"[i] = DFN_offense[j, :Actual_FP] 
            end
        end
    end
    

    return offense
end

In [ ]:
offense = clean_print_teams(cOffense, 4)

In [ ]:
current_offense = get_stripped_offense(4);


In [ ]:
writedlm( "dk_ids_20.csv",  print_team, ',')

# late swap

In [ ]:
function lateSwapTeamX(num, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games)#num is what team it is from 1-100, X is current teams
    m1 = Model(SCIP.Optimizer)
    set_silent(m1)
    #GLPK.Optimizer
    M = 100
    max_overlap = 5.1
    max_same_player = 40
    @variable(m1, x[1:cNum_offense], Bin) #pick players
    #@variable(m1, def[1:cNum_teams], Bin) #pick players
    #@variable(m1, PB[1:cNum_offense], Bin) #player boost
    
    
    #@constraint(m1, [i=1:length(FLEX_i)], PB[FLEX_i[i]] <= sum(x[opp_team[i][j]] for j = 1:length(opp_team[i])))
    
    #@constraint(m1, [i=1:length(QB_i)], PB[QB_i[i]] == 0)
    #@constraint(m1, [i=1:length(DST_i)], PB[DST_i[i]] == 0)

    #@variable(m1, G[1:num_games], Bin)
    #@constraint(m1,[i=1:num_games], 1.5 * G[i] <= sum(x[game_stack[i][j]] for j=1:length(game_stack[i])))
    #@constraint(m1, sum(G[i] for i=1:num_games) >= 2)
    #need to make num_games and game_stack, try using uniques to get it
    
    @variable(m1, u) #pick players


    @objective(m1, Max, u)

    @constraint(m1, sum(x[i]*rand_vec[i] for i=1:cNum_offense) +  # cOffense[i, :Projection] for i=1:cNum_offense) + 
                    (sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) * 9)>= u)
                    #sum(0.1 * x[i] * x[opp_def[i][j]] for i=1:cNum_offense for j=1:length(opp_def[i])) +
                    #sum(0.1*PB[i]*cOffense[i, :Projection] for i=1:cNum_offense) +
                    

    @constraint(m1, sum(x[QB_i[i]] for i=1:num_QB) == 1)
    @constraint(m1, sum(x[RB_i[i]] for i=1:num_RB) >= 2)
    @constraint(m1, sum(x[WR_i[i]] for i=1:num_WR) >= 3)
    @constraint(m1, sum(x[TE_i[i]] for i=1:num_TE) >= 1)
    @constraint(m1, sum(x[DST_i[i]] for i=1:num_DST) == 1)
    @constraint(m1, sum(x[FLEX_i[i]] for i=1:num_FLEX) == 7)
    @constraint(m1,sum(x[i]*cOffense[i, :Salary] for i=1:cNum_offense) <= 50000)
    @constraint(m1, sum(x[i] for i=1:cNum_offense) == 9)
                        
    @constraint(m1,sum(x[i]*cOffense."DK Ownership"[i] for i=1:cNum_offense) <= 50)
                        
    for i=1:cNum_offense
        #println(i)
        if (cOffense[i, :Team] in late_games) == false
            if isempty(current_teams[num]) == false
                #println(num)
                if i in current_teams[num]
                    #println(current_teams[num])


                    @constraint(m1, x[i] == 1)
                else
                    @constraint(m1, x[i] == 0)            
                end
            end
        end
    end


    
    
    #@constraint(m1, sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) >= 1.5)
    
    #bring back
                    
    @constraint(m1,[i=1:length(QB_i)], x[QB_i[i]] - sum(x[bring_back[i][j]] for j = 1:length(bring_back[i])) <= 0)
                        
    #team stack          
    @constraint(m1,[i=1:num_teams], sum(x[team_stack[i][j]] for j=1:length(team_stack[i])) <= 4)


    
    #@constraint(m1, sum(def[i] for i=1:cNum_teams) == 1)


    if num != 1
        #for i=1:num
        @constraint(m1,[i=1:num-1], sum(t[X[i][k],team_idx_cross[X[i][k]][j]] * x[team_idx_cross[X[i][k]][j]] for k = 1:9 for j=1:length(team_idx_cross[X[i][k]])) <= 7.9)
        #end
                        
        #@constraint(m1,[i=1:num-1], sum(x[X[i][k]] for k = 1:8) <= max_overlap)


    end 
                                        
    optimize!(m1)
    return(JuMP.value.(x))
end

In [ ]:
function lateSwapAllTeams(count, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, current_teams, late_games) #count is how many teams you want
    
    rand_vec = rand(cNum_offense,1)/25 .+1
    temp = cOffense[:, :Projection]
    rand_vec = rand_vec .* temp
    x1 = lateSwapTeamX(1, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games);

    B = Vector{Int}()
    for i=1:length(x1)
        if x1[i] > 0.5
            append!(B,i)
        end
    end
    X = push!(X, B)
    
    for i = 2:count
        rand_vec = rand(cNum_offense,1)/25 .+1
        temp = cOffense[:, :Projection]
        rand_vec = rand_vec .* temp
        @time x1 = lateSwapTeamX(i, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games);
        B = Vector{Int}()
        for i=1:length(x1)
            if x1[i] > 0.5
                append!(B,i)
            end
        end
        
        X = push!(X, B)
        #print(X)
        #D = hcat(D, d1)
        
        println(i)

    end

    return(X)
end

In [ ]:
X = Vector{Vector{Int}}()

@time all_teams = lateSwapAllTeams(110, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, a, late_teams);

In [ ]:
a[1]

In [ ]:
cOffense[2,:]

In [ ]:
a

In [ ]:
    ids = readdlm(string("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 12/dk_ids3.csv"), ',', Int)



In [ ]:
offense

In [ ]:
string(offense[all_teams[1][1],2])

In [ ]:
z = late_swap_print(all_teams, 12, 110, late_teams, cOffense)

In [ ]:
cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 12/")



writedlm( "ls_names.csv",  z, ',')

In [ ]:
function late_swap_print(new_teams, week, num_teams, late_teams, cOffense)

    num_teams_making = num_teams
    num = 9 #column

    #5 = salary
    #6 = regular projection
    #7 = DK value
    #8 = ownership
    #9 = DK ID
    #10 = ceiling projection
    
    ids = readdlm(string("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week ", week, "/dk_ids3.csv"), ',', Int)





    offense = cOffense
    lineup2 = ""
    #outfile = open(path_to_output, "w")
    print_team = []



    for i = 1:num_teams_making
        lineup = ["" "" "" "" "" "" "" "" ""]
        skip = []


        for j =1:9
            #println(i, " ", j)
            #if in ids, assign the same, else skip and come back to later
            in_old_teams = 0
            for t = 1:9
                #println(string(offense[new_teams[i][j],9]), ids[i,j])
                if string(offense[new_teams[i][j],9]) == string(ids[t,j]) && string(offense[new_teams[i][j],2]) in late_teams == false
                    #print("here")
                    lineup[t] = string(offense[new_teams[i][j],9])
                    in_old_teams = 0
                end
            end
            
            if in_old_teams == 0
                skip = push!(skip, new_teams[i][j])
            end
        end
    #println(skip)
        if length(skip) > 0
            for k in skip
            
                    if k in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[k,num])
                        end
                    elseif k in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[k,num])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[k,num])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[k,num])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[k,num])
                        end
                    else
                        println(i, " ", j, " sad boi")
                    end
                #end

            end
        end

       print_team = push!(print_team, lineup)
    end
    
    return print_team
end